# **Programação matemática**

**Alunos:**
- Heitor Rodrigues Sabino
- Ranulfo Mascari Neto

*Ao fazer, fiz no caderno o meu pensamento e tentei ser o mais simplista na minha resolução, visto que clareza é um ponto a ser avaliado*

## **Instalação e importação das bibliotecas**

In [ ]:
!pip install pulp
import pulp

## **Iniciando modelo de minimização**

In [ ]:
model = pulp.LpProblem('producao_camisetas', pulp.LpMinimize)

## **Variáveis de decisão**

In [5]:
x1 = pulp.LpVariable('x1', lowBound=0)  # Produção semana 1
x2 = pulp.LpVariable('x2', lowBound=0)  # Produção semana 2
x3 = pulp.LpVariable('x3', lowBound=0)  # Produção semana 3
x4 = pulp.LpVariable('x4', lowBound=0)  # Produção semana 4

y1 = pulp.LpVariable('y1', lowBound=0)  # Horas extras semana 1
y2 = pulp.LpVariable('y2', lowBound=0)  # Horas extras semana 2

s1 = pulp.LpVariable('s1', lowBound=0)  # Estoque semana 1
s2 = pulp.LpVariable('s2', lowBound=0)  # Estoque semana 2
s3 = pulp.LpVariable('s3', lowBound=0)  # Estoque semana 3

As variáveis de decisão representam as quantidades de camisetas produzidas, horas extras utilizadas e estoque ao final de cada semana:
- `x1, x2, x3, x4`: quantidade de camisetas produzidas nas semanas 1, 2, 3 e 4, respectivamente.
- `y1, y2`: quantidade de camisetas produzidas em **horas extras** nas semanas 1 e 2, respectivamente.
- `s1, s2, s3`: estoque de camisetas ao final das semanas 1, 2 e 3, respectivamente.

Essas variáveis são definidas com o argumento `lowBound=0`, garantindo que os valores serão **não negativos** (não podemos ter produção ou estoque negativos).

## **Função Objetivo**

In [6]:
model += (2 * x1 + 2.8 * y1 + 2 * x2 + 2.8 * y2 +
          2.5 * x3 + 2.5 * x4 +
          0.2 * s1 + 0.2 * s2 + 0.2 * s3), "Total_Cost"

A função objetivo representa o **custo total** que queremos minimizar. Ela inclui:
- Custo de produção regular:
  - Semana 1 e 2: R\$ 2,00 por camiseta.
  - Semana 3 e 4: R\$ 2,50 por camiseta.
- Custo de produção em horas extras:
  - Semana 1 e 2: R\$ 2,80 por camiseta.
- Custo de estocagem:
  - Custo de R\$ 0,20 por unidade estocada ao final das semanas 1, 2 e 3.

## **Restrições de prodrução**

In [7]:
model += x1 <= 25000, "Capacidade_semanal_1"
model += x2 <= 25000, "Capacidade_semanal_2"
model += x3 <= 25000, "Capacidade_semanal_3"
model += x4 <= 25000, "Capacidade_semanal_4"
model += y1 <= 10000, "Horas_extras_1"
model += y2 <= 10000, "Horas_extras_2"

Essas restrições garantem que:
- A fábrica pode produzir no máximo **25.000 camisetas** por semana, durante as 4 semanas.
- A fábrica pode fabricar até **10.000 camisetas em horas extras** nas semanas 1 e 2.

## **Restrições de estoque e demanda**

In [8]:
model += s1 == x1 + y1 - 5000, "Estoque_final_1"
model += s2 == x2 + y2 + s1 - 10000, "Estoque_final_2"
model += s3 == x3 + s2 - 30000, "Estoque_final_3"
model += x4 + s3 == 60000, "Atendimento_demanda_4"

Estas restrições controlam o fluxo de produção e estoque entre as semanas:
- **Demanda por camisetas** em cada semana:
  - Semana 1: 5.000 unidades.
  - Semana 2: 10.000 unidades.
  - Semana 3: 30.000 unidades.
  - Semana 4: 60.000 unidades.
- **Estoque** ao final de cada semana:
  - Semana 1: O estoque é igual à produção da semana 1 (`x1 + y1`) menos a demanda de 5.000 camisetas.
  - Semana 2: O estoque é o que sobrou da semana anterior (`s1`), somado à produção da semana 2, menos a demanda de 10.000.
  - Semana 3: Similar ao anterior, ajustando a demanda para 30.000 unidades.
  - Semana 4: A produção da semana 4 mais o estoque remanescente da semana 3 deve atender completamente a demanda de 60.000 unidades (com o estoque final sendo 0).

## **Solve**

In [9]:
model.solve()

1

## **Exibindo solução**

In [10]:
print("Status:", pulp.LpStatus[model.status])
print("Custo total: R$", pulp.value(model.objective))
print("Produção semana 1:", x1.varValue)
print("Produção horas extras semana 1:", y1.varValue)
print("Produção semana 2:", x2.varValue)
print("Produção horas extras semana 2:", y2.varValue)
print("Produção semana 3:", x3.varValue)
print("Produção semana 4:", x4.varValue)
print("Estoque semana 1:", s1.varValue)
print("Estoque semana 2:", s2.varValue)
print("Estoque semana 3:", s3.varValue)

Status: Optimal
Custo total: R$ 258000.0
Produção semana 1: 25000.0
Produção horas extras semana 1: 0.0
Produção semana 2: 25000.0
Produção horas extras semana 2: 5000.0
Produção semana 3: 25000.0
Produção semana 4: 25000.0
Estoque semana 1: 20000.0
Estoque semana 2: 40000.0
Estoque semana 3: 35000.0
